In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('./data/agnews/train.csv')

In [ ]:
df

In [ ]:
# show the first 10 titles and descriptions in the dataset full text
for i in range(3):
    title = df['Title'][i]
    description = df['Description'][i]
    label = df['Class Index'][i]
    num_tokens = description.count(' ') + 1
    print('Title: ', title)
    print('Description: ', description)
    print('Number of tokens: ', num_tokens)
    print('Label: ', label)

    print('===========================================')

In [ ]:
tokens = []
for i in range(len(df)):    
    description = df['Description'][i]    
    num_tokens = description.count(' ') + 1
    tokens.append(num_tokens)
df['tokens'] = tokens

In [ ]:
df.describe()

In [ ]:

df = df[['Class Index', 'Description']]

In [ ]:
df

In [ ]:
train_df = pd.read_csv('./data/agnews/train.csv')
test_df = pd.read_csv('./data/agnews/test.csv')

train_df = train_df[['Class Index', 'Description']]
test_df = test_df[['Class Index', 'Description']]



In [ ]:
import numpy as np

np.savetxt('./data/agnews/train.txt', train_df.values, fmt='%s', delimiter='\t')
np.savetxt('./data/agnews/test.txt', test_df.values, fmt='%s', delimiter='\t')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
kaggle_med = pd.read_csv('./data/original/kaggle_med/train.txt',delimiter='\t', header=None, names=['class', 'text'])
kaggle_med['text'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[1]) 
kaggle_med['class'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[0])

In [ ]:
kaggle_med['class'].unique()

In [ ]:
kaggle_med.describe()

In [ ]:
kaggle_med['class'].value_counts()

In [ ]:
kaggle_med.text.loc[kaggle_med['class'] == '7.']

In [ ]:
kaggle_med.text.loc[kaggle_med['class'] == '6.']

In [ ]:
kaggle_med.text.loc[kaggle_med['class'] == '0']

In [ ]:
kaggle_med.text.loc[kaggle_med['class'] == 'and']

In [ ]:
new_df_train = kaggle_med.drop(kaggle_med[(kaggle_med['class'] == 'and') | (kaggle_med['class'] == '6.') | (kaggle_med['class'] == '7.')].index)


In [ ]:
new_df_train.describe()

In [ ]:
np.savetxt('./data/original/kaggle_med/train.txt', new_df_train.values, fmt='%s', delimiter='\t')

In [ ]:
kaggle_med = pd.read_csv('./data/original/kaggle_med/test.txt',delimiter='\t', header=None, names=['class', 'text'])
kaggle_med['text'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[1]) 
kaggle_med['class'] = kaggle_med['class'].apply(lambda x: x.split(' ',1)[0])

In [ ]:
kaggle_med['class'].unique()

In [ ]:
kaggle_med.describe()

In [ ]:
df = pd.read_csv('./data/original/kaggle_med/train.txt',delimiter='\t', header=None, names=['class', 'text'])

In [ ]:
df

In [ ]:
duplicates = df[df.duplicated()]
len(duplicates)

In [ ]:
print(duplicates)

In [ ]:
duplicates = df[df['text'].duplicated()]
len(duplicates)

In [ ]:
for i in range(len(duplicates)):
    print(duplicates['text'].iloc[i])
    print('===========================================')

In [ ]:
duplicates = df.groupby('text').apply(lambda x: pd.Series({
    'class': list(x['class']),
    'count': len(x)
})).reset_index()
duplicates = duplicates[duplicates.duplicated(subset='class')]
for i in range(len(duplicates)):
    count = duplicates['count'].iloc[i]
    if count > 2:
        print('text: ',duplicates['text'].iloc[i])
        print('class: ',duplicates['class'].iloc[i])
        print('count: ',duplicates['count'].iloc[i])
        print('===========================================')

In [ ]:
df = pd.read_csv('./data/original/cardio/train.txt',delimiter='\t', header=None, names=['class', 'text'])
duplicates = df[df.duplicated()]
len(duplicates)

## new session
### augmentation

In [ ]:
import numpy as np
import pandas as pd
import textattack
from functions import *

In [ ]:
# Define the input paths for the datasets
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'yelp': './data/original/yelp/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',    
    'wordnet': 'wordnet_augmenter'
}


In [ ]:
# import snoop
# @snoop(depth=2)
def aug_samples(dataset_name, method_names, fraction, pct_words_to_swap, transformations_per_example, include_original):
    data = load_data(dict_datasets[dataset_name])
    print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')
    #data = data.sample(frac=fraction)
    #print(f'Sampled {len(data)} rows with fraction={fraction}')
    for method in method_names:
        if method not in dict_methods:
            raise ValueError(f"Method '{method}' is not supported")
        augmented_data = augment_text(
            data,
            dict_methods[method],
            fraction=fraction,
            pct_words_to_swap=pct_words_to_swap,
            transformations_per_example=transformations_per_example,
            label_column='class',
            target_column='text',
            include_original=include_original
        )
        augmented_data = augmented_data[['class', 'text']]
        np.savetxt(
            f'./data/augmented/{dataset_name}/{fraction}_{method}_{transformations_per_example}.txt',
            augmented_data.values,
            fmt='%s',
            delimiter='\t'
        )
        print(f'number of augmented samples: {len(augmented_data)}')

In [ ]:
aug_samples('agnews', ['eda'], 0.001, 0.5, 2, False)

In [ ]:
def aug_samples(dataset_name, method_names, fraction, pct_words_to_swap, transformations_per_example, include_original):
    data = load_data(dict_datasets[dataset_name])
    methods = method_names
    for method in methods:
        augmented_data = augment_text( data, method, fraction=fraction, pct_words_to_swap=pct_words_to_swap, transformations_per_example=transformations_per_example,
                                                            label_column='class', target_column='text', include_original=include_original)
        augmented_data = augmented_data[['class','text']]
        np.savetxt(f'./data/augmented/{dataset_name}/{fraction}_{method}_{transformations_per_example}.txt', augmented_data.values, fmt='%s', delimiter='\t')

In [ ]:
aug_samples('pc', ['checklist_augmenter'], 0.001, 0.5, 2, True)

# test

In [ ]:
data = load_data(dict_datasets['pc'])
df = data.copy( )

In [ ]:
df= df[~(df['text'].isin(['','none','nan','null','NaN','NULL','Null','None','NONE']))].reset_index(drop=True)

In [ ]:
np.savetxt('./data/original/pc/train_clean.txt', df.values, fmt='%s', delimiter='\t')

In [ ]:
aug_samples('pc_clean', ['checklist'], 0.001, 0.5, 2, True)

In [ ]:
df = pd.read_csv('./data/original/yelp/yelp.csv')
df = df[['stars', 'text']]
for i in range(len(df)):
    df.loc[i, "text"] = re.sub(r"[\\']", "", df.loc[i, "text"])

In [ ]:
import re

def clean_text(text):    
    text = re.sub(r"\n+", " ", text)
    text = re.sub(r"\t+", " ", text)
    text = re.sub(r"\r+", " ", text)
    text = re.sub(r"/\sThe+", "The", text)
    text = re.sub(r"[\\']+", "'", text)
    
    return text

In [ ]:
from sklearn.model_selection import train_test_split
import re
train, test = train_test_split(df, test_size=0.1, random_state=42)
train.to_csv('./data/original/yelp/train.csv', index=False)
test.to_csv('./data/original/yelp/test.csv', index=False)

test['text'] = test['text'].apply(lambda x: clean_text(x))
train['text'] = train['text'].apply(lambda x: clean_text(x))



np.savetxt('./data/original/yelp/train.txt', train.values, fmt='%s', delimiter='\t')
np.savetxt('./data/original/yelp/test.txt', test.values, fmt='%s', delimiter='\t')

In [ ]:
pd.read_csv('./data/original/yelp/train.txt',delimiter='\t', header=None, names=['class', 'text'])

## new session
### augmentation

In [ ]:
# Define the input paths for the datasets
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'kaggle_med': './data/original/kaggle_med/train.txt',
    'yelp': './data/original/yelp/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',
    'embedding': 'embedding_augmenter',
    'deletion': 'deletion_augmenter',
    'wordnet': 'wordnet_augmenter'
}

In [ ]:
def aug_samples(dataset_name, method_names, fraction, pct_words_to_swap, transformations_per_example, include_original):
    path = dict_datasets[dataset_name]
    data = pd.read_csv(path, delimiter='\t', header=None, names=['class', 'text'])
    data = data.sample(frac=fraction)
    print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')
    #data = data.sample(frac=fraction)
    #print(f'Sampled {len(data)} rows with fraction={fraction}')
    for method in method_names:
        if method not in dict_methods:
            raise ValueError(f"Method '{method}' is not supported")
        augmented_data = augment_text(
            data,
            dict_methods[method],
            fraction=1,
            pct_words_to_swap=pct_words_to_swap,
            transformations_per_example=transformations_per_example,
            label_column='class',
            target_column='text',
            include_original=include_original
        )
        augmented_data = augmented_data[['class', 'text']]
        np.savetxt(
            f'./data/augmented/{dataset_name}/{fraction}_{method}_{transformations_per_example}.txt',
            augmented_data.values,
            fmt='%s',
            delimiter='\t'
        )
        print(f'number of augmented samples: {len(augmented_data)}')

In [ ]:
aug_samples('yelp', ['eda'], 0.001, 0.5, 2, False)

In [ ]:
df = pd.read_csv('./data/original/yelp/train.txt',delimiter='\t', header=None, names=['class', 'text'])

In [ ]:
df.iloc[4284,1]

In [ ]:
import numpy as np
import pandas as pd
import textattack
from functions1 import *
method_names = ['eda']
dataset_name = 'yelp'
fraction = 0.001
path = dict_datasets[dataset_name]
data = pd.read_csv(path, delimiter='\t', header=None, names=['class', 'text'])
data = data.sample(frac=fraction)
print(f'Loaded {len(data)} rows of data from {dict_datasets[dataset_name]}')

In [ ]:
# Define the input paths for the datasets
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'kaggle_med': './data/original/kaggle_med/train.txt',
    'yelp': './data/original/yelp/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',
    'embedding': 'embedding_augmenter',
    'deletion': 'deletion_augmenter',
    'wordnet': 'wordnet_augmenter'
}

import numpy as np
import pandas as pd
import textattack
from functions import *
def aug_samples(name,frac):
    path = dict_datasets[name]
    data = load_data(path)
    data = data.sample(frac=frac)
    print(f'Loaded {len(data)} rows of data from {name}')
    #methods = ['eda_augmenter','wordnet_augmenter','aeda_augmenter','backtranslation_augmenter']
    methods = ['checklist_augmenter']
    for method in methods:
        augmented_data = augment_text(data, method,fraction=1,pct_words_to_swap=0.2 ,transformations_per_example=1,
                    label_column='class',target_column='text',include_original=True)
        augmented_data = augmented_data[['class','text']]
        np.savetxt(f'./data/augmented/{name}/{method}_aug.txt', augmented_data.values, fmt='%s', delimiter='\t')

In [ ]:
aug_samples('pc',0.001)

In [ ]:
import numpy as np
import pandas as pd
import textattack
import logging
from functions import *

# Define the input paths for the datasets
dict_datasets = {
    'agnews': './data/original/agnews/train.txt',
    'cardio': './data/original/cardio/train.txt',
    'cr': './data/original/cr/train.txt',
    'pc': './data/original/pc/train.txt',
    'pc_clean': './data/original/pc/train_clean.txt',
    'sst2': './data/original/sst2/train.txt',
    'subj': './data/original/subj/train.txt',
    'trec': './data/original/trec/train.txt',
    'kaggle_med': './data/original/kaggle_med/train.txt',
    'yelp': './data/original/yelp/train.txt'
}

# Define the augmentation methods and their names
dict_methods = {
    'aeda': 'aeda_augmenter',
    'backtranslation': 'backtranslation_augmenter',
    'checklist': 'checklist_augmenter',
    'clare': 'clare_augmenter',
    'eda': 'eda_augmenter',
    'wordnet': 'wordnet_augmenter'
}

aug_num = [1,2,4]
aug_frac = [0.1,0.2,0.5,1]

logging.basicConfig(filename='augment.log', filemode='a', format='%(asctime)s - %(levelname)s - %(message)s', level=logging.INFO)

def aug_samples(name, frac):
    path = dict_datasets[name]
    data = load_data(path)
    try:
        data = data.sample(frac=frac)
    except:
        data = data.sample(n=1)
    logging.info(f'Loaded {len(data)} rows of data from {name}')
    
    #methods = ['eda_augmenter','wordnet_augmenter','aeda_augmenter','backtranslation_augmenter']
    methods = ['checklist_augmenter', 'eda_augmenter', 'wordnet_augmenter', 'aeda_augmenter', 'backtranslation_augmenter', 'clare_augmenter']
    
    for method in methods:
        augmented_data = augment_text(data, method, fraction=1, pct_words_to_swap=0.2, transformations_per_example=1,
                                      label_column='class', target_column='text', include_original=True)
        augmented_data = augmented_data[['class','text']]
        np.savetxt(f'./data/augmented/{name}/{method}_aug.txt', augmented_data.values, fmt='%s', delimiter='\t')
        logging.info(f'Augmented {len(augmented_data)} rows of data from {name} using {method} method')


In [ ]:
aug_samples('yelp', 0.0001)

In [ ]:
#path = dict_datasets['yelp']
path = './data/original/yelp/train.csv'
df = load_data(path)
#df = pd.read_csv(path,  names=['text'])

In [ ]:
df 

In [ ]:
df['class'] = df['text'].apply(lambda x: x.split('\t')[0])

In [ ]:
df

In [ ]:
df['text'].iloc[1]

In [ ]:
df['text'].iloc[0]


In [ ]:
for i in range(10):
    print(df['text'].iloc[i])

In [ ]:
df

In [ ]:
import pandas as pd

def load_data(path):
    """
    Loads data from a txt file.
    """
    # check file format
    if path.endswith('.txt'):
        try:
            df = pd.read_csv(path, sep='|', header=None, names=['text'])
        except:
            df = pd.read_csv(path, sep='\t', header=None, names=['text'])
        df['class'], df['text'] = zip(*df['text'].apply(lambda x: x.split('\t', 1)))
    else:
        df = pd.read_csv(path, header=None, names=['class', 'text'])

    return df[['class', 'text']]


In [ ]:
path = './data/original/yelp/train.csv'
df = load_data(path)

In [ ]:
df

In [ ]:
import pandas as pd

def load_data(path):
    """
    Loads data from a txt file.
    """
    # check file format
    if path.endswith('.txt'):
        try:
            df = pd.read_csv(path, sep='|', header=None, names=['text'])
        except:
            try:
                df = pd.read_csv(path, sep='\t', header=None, names=['text'])
            except:
                df = pd.read_csv(path, header=None, names=['text'])
                df['class'] = df['text'].apply(lambda x: x.split(' ',1)[0])
                df['text'] = df['text'].apply(lambda x: x.split(' ',1)[1])
        else:
            df['class'], df['text'] = zip(*df['text'].apply(lambda x: x.split('\t', 1)))
    else:
        df = pd.read_csv(path, header=None, names=['class', 'text'],skiprows=1)

    return df[['class', 'text']]


In [ ]:
path = './data/original/bbc/train.csv'
df = load_data(path)

In [ ]:
df

In [6]:
from sklearn.model_selection import train_test_split
import re
import pandas as pd
import numpy as np

In [7]:
# bbc = pd.read_csv('./data/original/bbc/bbc.csv', header=None, names=['class', 'text'],skiprows=1)
# bbc['class'] = bbc['class'].map({'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4})
# train, test = train_test_split(bbc, test_size=0.1, random_state=42)

# train.to_csv('./data/original/bbc/train.csv', index=False)
# test.to_csv('./data/original/bbc/test.csv', index=False)

# np.savetxt('./data/original/bbc/train.txt', train.values, fmt='%s', delimiter='\t')
# np.savetxt('./data/original/bbc/test.txt', test.values, fmt='%s', delimiter='\t')

In [9]:
test = pd.read_csv('./data/original/bbc/test.csv')

In [10]:
test

,class,text
0,2,brown and blair face new rift claims for the u...
1,0,small firms hit by rising costs rising fuel ...
2,1,spirit awards hail sideways the comedy sideway...
3,4,microsoft releases patches microsoft has warne...
4,3,arsenal through on penalties arsenal win 4-2 o...
...,...,...
218,3,pearce keen on succeeding keegan joint assista...
219,2,blair blasts tory spending plans tony blair ha...
220,3,moya fights back for indian title carlos moya ...
221,2,crucial decision on super-casinos a decision o...


In [ ]:
train_df['class'] = train_df['Category'].map({'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4})
test_df['class'] = test_df['Category'].map({'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4})

# train_df = train_df[['class', 'Text']]
# test_df = test_df[['class', 'Text']]

In [ ]:
train_df

In [ ]:
test_df

In [16]:

def clean_text(text):    
    text = re.sub(r"\n+", " ", text)
    text = re.sub(r"\t+", " ", text)
    text = re.sub(r"\r+", " ", text)
    text = re.sub(r"/\sThe+", "The", text)
    #text = re.sub(r"(\\')+", "'", text)
    text = re.sub(r'\\+', '', text)
    
    return text

In [20]:
yelp = pd.read_csv('./data/original/yelp/yelp.csv')
yelp = yelp[['stars', 'text']]
yelp['text'] = yelp['text'].map(clean_text)
yelp.columns = ['class', 'text']
yelp['text'] = yelp['text'].apply(lambda x: str(x).replace("\\'s","'s"))

In [13]:
yelp

,class,text
0,5,My wife took me here on my birthday for breakf...
1,5,I have no idea why some people give bad review...
2,4,love the gyro plate. Rice is so good and I als...
3,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!..."
4,5,General Manager Scott Petello is a good egg!!!...
...,...,...
9995,3,First visit...Had lunch here today - used my G...
9996,4,Should be called house of deliciousness! I cou...
9997,4,I recently visited Olive and Ivy for business ...
9998,2,My nephew just moved to Scottsdale recently so...


In [21]:
train, test = train_test_split(yelp, test_size=0.1, random_state=42)
train.to_csv('./data/original/bbc/train.csv', index=False)
test.to_csv('./data/original/bbc/test.csv', index=False)

np.savetxt('./data/original/bbc/train.txt', train.values, fmt='%s', delimiter='\t')
np.savetxt('./data/original/bbc/test.txt', test.values, fmt='%s', delimiter='\t')

In [ ]:
for i in range (10):
    print(yelp['class'].iloc[i])

In [22]:
train['text'].iloc[2248]

'The upstairs theater is nice enough. There are a few nice semi-upscale restaurants here. It\'s also got a really good swanky (and somewhat hidden) bar here.  So why only 2 stars? Well, because it smells like this is where all the diarrhea in the world goes to die... especially the parking structure. If you can escape the smell though, this is a nice spot for a date. The problem with that is when you go walk her to her car, you have to try and figure out how to have a good night kiss while your nose is telling your brain that you are swimming in a septic tank. The new Waterfront mall corridor stinks like this as well. Maybe that\'s where the term "filthy rich" comes from. What do these people eat?'

In [ ]:
txt = 'There are a few nice semi-upscale restaurants here. It\'s also got a really good swanky (and somewhat hidden) bar here.'
new_txt  = clean_text(txt)

In [ ]:
new_txt